In [3]:
import theano
import theano.tensor as T
from theano.tensor.shared_randomstreams import RandomStreams
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
 
#define the ode function
#dc/dt  = f(c, lambda)
#c is a vector with n components
def evolve(c, n, k, l):
	hill = T.pow(k,n)/(T.pow(c, n)+T.pow(k,n))
	rep = T.roll(hill, 1, axis=1)
	return rep - l*c
 
def euler(c, n, k, l, dt):
	return T.cast(c + dt*evolve(c, n, k, l) + T.sqrt(dt)*c*rv_n, 'float32')
 
def rk4(c, n, k, l, dt):
	'''
	Adapted from
	http://people.sc.fsu.edu/~jburkardt/c_src/stochastic_rk/stochastic_rk.html
	'''
	a21 =   2.71644396264860
	a31 = - 6.95653259006152
	a32 =   0.78313689457981
	a41 =   0.0
	a42 =   0.48257353309214
	a43 =   0.26171080165848
	a51 =   0.47012396888046
	a52 =   0.36597075368373
	a53 =   0.08906615686702
	a54 =   0.07483912056879
 
	q1 =   2.12709852335625
	q2 =   2.73245878238737
	q3 =  11.22760917474960
	q4 =  13.36199560336697
 
	x1 = c
	k1 = dt * evolve(x1, n, k, l) + T.sqrt(dt) * c * rv_n
 
	x2 = x1 + a21 * k1
	k2 = dt * evolve(x2, n, k, l) + T.sqrt(dt) * c * rv_n
 
	x3 = x1 + a31 * k1 + a32 * k2
	k3 = dt * evolve(x3, n, k, l) + T.sqrt(dt) * c * rv_n
 
	x4 = x1 + a41 * k1 + a42 * k2
	k4 = dt * evolve(x4, n, k, l) + T.sqrt(dt) * c * rv_n
 
	return T.cast(x1 + a51 * k1 + a52 * k2 + a53 * k3 + a54 * k4, 'float32')
 
if __name__ == '__main__':
	#random
	srng = RandomStreams(seed=31415)
 
	#define symbolic variables
	dt = T.fscalar("dt")
	k = T.fscalar("k")
	l = T.fscalar("l")
	n = T.fscalar("n")
	c = T.fmatrix("c")
 
	#define numeric variables
	num_samples = 50000
	init = np.ones((num_samples, 3), dtype='float32')
	init[:, 1:3] = 0.2
	c0 = theano.shared(init)
	n0 = 6
	k0 = 0.5
	l0 = 1/(1+np.power(k0, n0))
	dt0 = 0.1
	total_time = 8
	total_steps = int(total_time/dt0)
	rv_n = srng.normal(c.shape, std=0.1) #is a shared variable
 
	#create loop
	#first symbolic loop with everything
	(cout, updates) = theano.scan(fn=rk4,
									outputs_info=[c], #output shape
									non_sequences=[n, k, l, dt], #fixed parameters
									n_steps=total_steps)
	#compile it
	sim = theano.function(inputs=[n, k, l, dt], 
						outputs=cout, 
						givens={c:c0}, 
						updates=updates,
						allow_input_downcast=True)
 
	print ("running sim...")
	start = time.clock()
	cout = sim(n0, k0, l0, dt0)
	diff = (time.clock() - start)
	print ("done in", diff, "s at ", diff/num_samples, "s per path")
 
	downsample_factor_t = 0.1/dt0 #always show 10 points per time unit
	downsample_factor_p = num_samples/50
 
	x = np.linspace(0, total_time, total_steps/downsample_factor_t)
	gs = gridspec.GridSpec(3, 2, width_ratios=[4,1])
 
	plt.subplot(gs[0, 0])
	plt.plot(x, cout[::downsample_factor_t, ::downsample_factor_p, 0])
	plt.subplot(gs[1, 0])
	plt.plot(x, cout[::downsample_factor_t, ::downsample_factor_p, 1])
	plt.subplot(gs[2, 0])
	plt.plot(x, cout[::downsample_factor_t, ::downsample_factor_p, 2])
 
	plt.subplot(gs[0, 1])
	plt.hist(cout[-1,:,0], 30, 
				normed=True, histtype='bar')
	plt.subplot(gs[1, 1])
	plt.hist(cout[-1,:,1], 30, 
				normed=True, histtype='bar')
	plt.subplot(gs[2, 1])
	plt.hist(cout[-1,:,2], 30, 
				normed=True, histtype='bar')
 
	#plt.show()
	plt.clf()
	bins = np.linspace(0.1 , 1, 50)
	for i in xrange(cout.shape[0]):
		plt.hist(cout[i,:,1], bins, 
				normed=True, histtype='bar')
		plt.xlim([0.1, 1])
		plt.ylim([0, 10])
		plt.savefig("pics/rep"+str(i)+".png")
		plt.clf()

ModuleNotFoundError: No module named 'theano'

In [4]:
class Repressilator:
    def __init__( self ):
        """ initialize repressilator simulation (stochastic simulation by
            default): initially one gene is activated and the other two are
            repressed
        """
        usestoch = True
        reactionstrs = [
            # Protein represses other gene's expression
            "2 pa + gb --> cb , k=1",
            "2 pb + gc --> cc , k=1",
            "2 pc + ga --> ca , k=1",
            "cb --> 2 pa + gb , k=1",
            "cc --> 2 pb + gc , k=1",
            "ca --> 2 pc + ga , k=1",
            
            # Transcription (gene -> mRNA):
            "ga --> ga + ma  , k=5",
            "gb --> gb + mb  , k=5",
            "gc --> gc + mc  , k=5",

            # Leaky transcription
            "ca --> ca + ma , k=0",
            "cb --> cb + mb , k=0",
            "cc --> cc + mc , k=0",

            # Translation (mRNA -> Protein):
            "ma --> ma + pa , k=1",
            "mb --> mb + pb , k=1",
            "mc --> mc + pc , k=1",

            # Decay of mRNA and Protein:
            "ma -->  ,  k=0.5",
            "mb -->  ,  k=0.5",
            "mc -->  ,  k=0.5",
            "pa -->  ,  k=0.1",
            "pb -->  ,  k=0.1",
            "pc -->  ,  k=0.1",
        ]

	self.reactor = GillespieVessel()
	self.reactor.parse(reactionstrs)

        nav = 100
        self.reactor.set_nav(nav)

        # initial conditions: one gene is activated, the other two are repressed
        self.reactor.inject('ga',  1 * nav)
        self.reactor.inject('cb', 1 * nav)
        self.reactor.inject('cc', 1 * nav)
        #self.reactor.trace()

    def run( self, finalvt=200 ):
        """ run the repressilator until the simulation time reaches finalvt """
        plotall = False
        self.reactor.trace_title()
        self.reactor.trace_mult()
        ptime = 0.0
        while (self.reactor.vtime() <= finalvt):
            t = self.reactor.vtime()
            self.reactor.iterate()
            dt = t - ptime
            if (dt >= 0.2 or plotall):
                self.reactor.trace_mult()
                ptime = t

if __name__ == '__main__':
    repress = Repressilator()
    repress.run()

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-4-b5b242a228bc>, line 41)

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

def oocyte(y,t):
    f= 40
    n= 5
    K= 20
    k= 0.2
    P= 0
    dydt= np.empty(len(y))
    dydt= k*p + f*y**n/(K**n + y**n) - y
    return dydt

init= [30]
t= np.arange(0,100,0.1)
y= odeint(oocyte, init, t) 

plt.figure() 
plt.plot(t, y[:,0], label= 'active Mos') 
plt.xlabel('time') 
plt.legend() 
plt.show() 


NameError: name 'p' is not defined

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [44]:
f=40
n=5
K=20
k=0.2
P=0

def oocyte(y,t):
        dydt= np.empty(len(y));
        dydt=(k*p)(+f*y**n/(K**n+y**n))-y;
        return dyd
init= [30]
t= np.arange(0,100,0.1)
y= odeint(oocyte, init, t) 

plt.figure() 
plt.plot(t,y) 
plt.show() 

NameError: name 'p' is not defined